In [7]:
 # We import necessary libraries: json forworking with JSON data, csv for handling CSV files, and datetime for manipulating dates and times. 
    
import json
import csv
from datetime import datetime


In [8]:
#The function load_data reads a JSON file, 
#parses each line as a JSON object, and returns the data as a list of dictionaries or lists in Python.
def load_data(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return data

In [9]:
#The function `convert_to_unix_timestamp` converts a datetime string to a Unix timestamp integer.
def convert_to_unix_timestamp(datetime_str):
    datetime_obj = datetime.fromisoformat(datetime_str)
    return int(datetime_obj.timestamp())

In [10]:
#The function process_conditions extracts patient conditions with timestamps based on corresponding encounter times, 
#storing them in a dictionary. The function write_to_csv writes the patient conditions data to a CSV file with specified columns.

def process_conditions(patient_data, condition_data, encounter_data):
    patient_conditions = {}
    for condition in condition_data:
        patient_id = condition['subject']['reference'].split('/')[-1]
        encounter_id = condition['encounter']['reference'].split('/')[-1]
        for encounter in encounter_data:
            if encounter_id == encounter['id']:
                encounter_time = encounter['period']['start']
                condition_time = convert_to_unix_timestamp(encounter_time)
                break
        if 'text' in condition['code']:
            description = condition['code']['text']
        else:
            description = "Unknown"
        if patient_id in patient_conditions:
            patient_conditions[patient_id].append({
                'timestamp': condition_time,
                'code': condition['code']['coding'][0]['code'],
                'description': description
            })
        else:
            patient_conditions[patient_id] = [{
                'timestamp': condition_time,
                'code': condition['code']['coding'][0]['code'],
                'description': description
            }]
    return patient_conditions

def write_to_csv(patient_conditions, output_file):
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['pid', 'time', 'code', 'description']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for patient_id, conditions in patient_conditions.items():
            for condition in conditions:
                writer.writerow({
                    'pid': patient_id,
                    'time': condition['timestamp'],
                    'code': condition['code'],
                    'description': condition['description']
                })


In [11]:
#The provided code loads patient, condition, and encounter data from JSON files, 
#processes the conditions with corresponding encounter timestamps, and writes the data to a CSV file.

patient_data = load_data("Patient.ndjson")
condition_data = load_data("Condition.ndjson")
encounter_data = load_data("Encounter.ndjson")

patient_conditions = process_conditions(patient_data, condition_data, encounter_data)

write_to_csv(patient_conditions, "patient_conditions.csv")
